In [1]:
import cv2
import mediapipe as mp
import math


In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
def distancia(p1, p2):
    return math.sqrt((p2.x - p1.x) ** 2 + (p2.y - p1.y) ** 2)

In [4]:
# funcion para determinar si un dedo esta levantado o no

# tip = punta
# pip = primera articulacion
# dip = segunda articulacion
# mcp = metacarpiana (base del dedo)
def dedo_levantado(landmarks, dedo_tip, dedo_dip, dedo_pip, dedo_mcp, umbral=0.1):
    # comparar la posición "y" del punto a la punta con los puntos anteriores
    return (
        landmarks[dedo_tip].y < landmarks[dedo_dip].y
        and landmarks[dedo_dip].y < landmarks[dedo_pip].y
        and landmarks[dedo_pip].y < landmarks[dedo_mcp].y
    )

In [5]:
ruta = "../dataSet/jpg/manos.jpg"

with mp_hands.Hands(
    static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5
) as hands:
    image = cv2.imread(ruta)
    height, width, _ = image.shape
    image = cv2.flip(image, 1)  # Voltear la imagen horizontalmente
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    print("Hadedness: ", results.multi_handedness)

Hadedness:  [classification {
  index: 0
  score: 0.986841381
  label: "Left"
}
, classification {
  index: 1
  score: 0.953673
  label: "Right"
}
]


In [6]:
if results.multi_hand_landmarks:
    for hands_landmarks in results.multi_hand_landmarks:
        # Dibujar las marcas y conexiones de la mano
        mp_drawing.draw_landmarks(image, hands_landmarks, mp_hands.HAND_CONNECTIONS)

        # Obtener las marcas como una lista
        landmarks = hands_landmarks.landmark

        # Definir los indices de los puntos de los dedos (segun Mediapipe)
        PULGAR_TIP = 4
        INDICE_TIP = 8
        MEDIO_TIP = 12
        ANULAR_TIP = 16
        MENIQUE_TIP = 20

        # Puntos para cada dedo (tip, dip, pip, mcp)
        dedos = {
            "Pulgar": (PULGAR_TIP, 2, 1, 0),
            "Indice": (INDICE_TIP, 6, 5, 0),
            "Medio": (MEDIO_TIP, 10, 9, 0),
            "Anular": (ANULAR_TIP, 14, 13, 0),
            "Menique": (MENIQUE_TIP, 18, 17, 0),
        }

        # Verificar que dedos están levantados
        dedos_levantados = []
        for nombre, (tip, dip, pip, mcp) in dedos.items():
            if dedo_levantado(landmarks, tip, dip, pip, mcp):
                dedos_levantados.append(nombre)
        print("Dedos levantados:", dedos_levantados)

Dedos levantados: ['Pulgar', 'Indice', 'Medio', 'Anular', 'Menique']
Dedos levantados: ['Pulgar', 'Indice', 'Medio', 'Anular', 'Menique']


In [7]:
# Dibujar las marcas y conexiones de ambas manos con color azul
for hands_landmarks in results.multi_hand_landmarks:
    mp_drawing.draw_landmarks(
        image, hands_landmarks, mp_hands.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),  # Puntos en blanco
        mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)  # Conexiones en azul
    )
mp_drawing.draw_landmarks(
    image, hands_landmarks, mp_hands.HAND_CONNECTIONS,
    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),  # Puntos en blanco
    mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)  # Conexiones en azul
)
cv2.putText(
    image,
    f"Dedos: {', '.join(dedos_levantados)}",
    (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX,
    1,
    (0, 255, 0),
    2,
)

array([[[21, 22, 18],
        [21, 22, 18],
        [22, 23, 19],
        ...,
        [16, 17, 13],
        [16, 17, 13],
        [16, 17, 13]],

       [[21, 22, 18],
        [21, 22, 18],
        [22, 23, 19],
        ...,
        [16, 17, 13],
        [16, 17, 13],
        [16, 17, 13]],

       [[22, 23, 19],
        [22, 23, 19],
        [21, 22, 18],
        ...,
        [16, 17, 13],
        [16, 17, 13],
        [16, 17, 13]],

       ...,

       [[12, 13,  9],
        [12, 13,  9],
        [12, 13,  9],
        ...,
        [11, 12,  8],
        [11, 12,  8],
        [11, 12,  8]],

       [[12, 13,  9],
        [12, 13,  9],
        [12, 13,  9],
        ...,
        [11, 12,  8],
        [11, 12,  8],
        [11, 12,  8]],

       [[12, 13,  9],
        [12, 13,  9],
        [12, 13,  9],
        ...,
        [11, 12,  8],
        [11, 12,  8],
        [11, 12,  8]]], dtype=uint8)

In [8]:
# Mostrar la imagen resultante 
# Redimensionar la imagen para que sea más pequeña
image_resized = cv2.resize(image, (width // 2, height // 2))
cv2.imshow("Imagen", image_resized)
cv2.waitKey(0) # Esperar a que se presione una tecla
cv2.destroyAllWindows() # Cerrar todas las ventanas